<a href="https://colab.research.google.com/github/ella00100/JAVA_basic/blob/main/Titanic_tuto01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Titanic 

#캐글과 colab 연결하기
1. kaggle에서 API token 다운
2. kaggle과 colab library 불러오기
3. file upload
4. json 파일 이동
5. 데이터 셋 불러오기 

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
!ls -lha kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json
# 내가 참가한 대회 리스트 확인
!kaggle competitions list

In [ ]:
!kaggle competitions download -c titanic

In [ ]:
!ls


In [ ]:
!unzip titanic.zip

라이브러리 불러오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)

import missingno as msno #null data를 쉽게 보여주는 library
import warnings
warnings.filterwarnings('ignore')

#plot를 새로운 창이 아니고 바로 볼 수 있도록 inline 사용
%matplotlib inline 

# 1. Data set 확인

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
#상위 5개 데이터 출력
df_train.head()

- 생존 예측 시스템 구축
- feature : Pclass, Age, Sibsp, Parch, Fare
- target label : Survived

In [ ]:
#feature가 가진 통계치 반환
df_train.describe()

# 1.1 NULL 데이터 확인

In [ ]:
#데이터의 null데이터 비중확인 
for col in df_train.columns:
  msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col,100*(df_train[col].isnull().sum() / df_train[col].shape[0]))
  print(msg)

#Age와 Cabin에 null 데이터 존재

In [ ]:
for col in df_test.columns:
  msg = 'column: {:10}\t Percent of NaN value: {:.2f}%'.format(col, 100*(df_test[col].isnull().sum() /df_test[col].shape[0]))
  print(msg)

#Age와 Cabin에 null 데이터 존재재

In [ ]:
#null 값의 분포포를 시각화 하여 보기
msno.matrix(df=df_train.iloc[:,:], figsize=(8,8), color = (0.2, 0.5, 0.2))

In [ ]:
msno.bar(df=df_train.iloc[:,:], figsize=(8,8), color = (0.7, 0.7, 0.3))

In [ ]:
msno.bar(df=df_test.iloc[:,:], figsize=(8,8), color = (0.8, 0.5, 0.2))

# 1.2 Target label 확인

In [ ]:
#frame과 subplot 생성 
f, ax = plt.subplots(1,2,figsize = (18,8)) #plt(1행, 2열, figsize)

#df_train의 생존 비율 원그래프 그리기
df_train['Survived'].value_counts().plot.pie(
    explode=[0,0.1], autopct="%1.1f%%",ax=ax[0], shadow=True)
#explode:파이 분리시키기, autopct: 형식, ax=첫번째 subplot, shadow: 그림자

ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')

#df_train의 생존 비율 countPlot 그리기
sns.countplot(x='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()


# 2. EDA(Exploratory data analysis)
데이터를 탐색하고 분석하는 과정

EDA는 데이터의 특성과 구조를 파악하고, 데이터의 분포와 상관관계를 조사하며, 데이터에 대한 이해도를 높이는 과정이다.

# 2.1 Pclass

- 서수형 data, 카테고리, 순서가 있는 data type

In [ ]:
df_train[['Pclass', 'Survived']]

In [ ]:
#Pclass에 따른 인원
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).count()

In [ ]:
#Pclass에 따른 생존인원
df_train[['Pclass','Survived']].groupby(['Pclass'], as_index=True).sum()

In [ ]:
#pandas를 이용하여 crosstab으로 출력
pd.crosstab(df_train['Pclass'], df_train['Survived'], 
            margins=True).style.background_gradient(cmap='gist_earth_r')

In [ ]:
#생존비를 막대 그래프로 출력
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True
                        ).mean().sort_values(by='Survived',ascending=False).plot.bar()

In [ ]:
y_position = 1.02
f, ax = plt.subplots(1, 2, figsize=(18,8))

#Pclass별 탑승객 인원 출력력
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32', '#FFDF00', '#D3D3D3'], ax=ax[0])
ax[0].set_title('Number of Passengers By Pclass', y=y_position)
ax[0].set_ylabel('Count')

#Pclass별 생존인원/사망인원 출력
sns.countplot(x='Pclass', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y=y_position)
plt.show()


- 첫번째 그래프 : 3등석의 탑승객이 1/2등석에 2배 이상임
- 두번째 그래프 : 탑승 class가 낮아질수록 생존확률도 낮아짐
- 3등석의 경우 생존비율이 특히 낮음


# 2.2 Sex

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18,8))

#성별 생존 비율 막대 그래프
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=True
                                      ).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')

#성별 생존자/사망자 수 
sns.countplot(x='Sex', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Sex: Survived vs Dead')
plt.show()

In [ ]:
#성별 생존 비율
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
pd.crosstab(df_train['Sex'], df_train['Survived'], margins=True).style.background_gradient(cmap='gist_gray_r')

# 2.3 Both Sex and Pclass

In [ ]:
#Pclass 별, 성별에 따른 생존 비율
sns.catplot(x='Pclass', y='Survived', hue='Sex', kind='point', data=df_train,  height=5, aspect=1.5)

In [ ]:
#각각의 Pclass의 성별 생존비율
sns.catplot(x='Sex', y='Survived', col='Pclass', kind='point', data=df_train,  height=5, aspect=1.5)

# 2.4 Age

In [ ]:
#탑승객의 최고/최소/평균 나이

print('최고령자 탑승객 : {:.1f} Years'.format(df_train['Age'].max()))
print('최연소자 탐승객 : {:.1f} Years'.format(df_train['Age'].min()))
print('탑승객 평균 나이 : {:.1f} Years'.format(df_train['Age'].mean()))

In [ ]:

#생존에 따른 Age의 histogram

fig, x = plt.subplots(1, 1, figsize=(9,5))
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax=x)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax=x)
plt.legend(['Survived == 1', 'Survived == 0']) #범례 추가
plt.show()


In [ ]:
#Pclass 별 나이 분포포 히스토그램

plt.figure(figsize=(8,6))
df_train['Age'][df_train['Pclass']==1].plot(kind='kde')
df_train['Age'][df_train['Pclass']==2].plot(kind='kde')
df_train['Age'][df_train['Pclass']==3].plot(kind='kde')

plt.xlabel('Age')
plt.title('Age Distribution within classes')
plt.legend(['1st Class', '2nd Class', '3rd Class'])

In [ ]:
#나이증가에 따른 생존율 변화

cummulate_survival_ratio =[]
for i in range(1,80): #나이를 80세까지 넓혀가며 탐색색
  cummulate_survival_ratio.append(
      df_train[df_train['Age']<i]['Survived'].sum()/
      len(df_train[df_train['Age']<i]['Survived']))
plt.figure(figsize=(7,7))
plt.plot(cummulate_survival_ratio)
plt.title('Sruvival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age(0~x)')
plt.show()

# 2.5 Pclass, Sex, Age

In [ ]:
f,ax= plt.subplots(1,2,figsize=(18,8))

#Pclass와 Age의 생존 분포
sns.violinplot(x="Pclass", y="Age", hue="Survived", data=df_train, scale='count', split=True, ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))

#sex와 Age의 생존 분보
sns.violinplot(x='Sex', y="Age", hue="Survived", data=df_train, scale='count', split=True, ax=ax[1])
ax[1].set_title("Sex and Age vs Survived")
ax[1].set_yticks(range(0,110,10))
plt.show()

# 2.6 Embarked
: 탑승한 항구

In [ ]:
f, ax = plt.subplots(1,1,figsize=(7,7))

#탑승한 항구에 따른 생존률률
df_train[['Embarked', 'Survived']].groupby(['Embarked'],as_index=True
                    ).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax)

In [ ]:
f,ax=plt.subplots(2,2,figsize=(20,15))

#탑승한 항구 별 탑승객 수
sns.countplot(x='Embarked', data=df_train, ax=ax[0,0])
ax[0,0].set_title('(1) No. Of Passengers Boarded')

#탑승한 항구에 따른 성별 탑승객 수
sns.countplot(x='Embarked', hue='Sex', data=df_train, ax=ax[0,1])
ax[0,1].set_title('(2) Male-Female Split for Embarked')

#탑승한 항구 별 생존 인원원
sns.countplot(x='Embarked', hue='Survived', data=df_train, ax=ax[1,0])
ax[1,0].set_title('(3) Embarked vs Survived')

#탑승한 항구에 따른른 Pclass별 탑승객 수 
sns.countplot(x='Embarked', hue='Pclass', data=df_train, ax=ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')

plt.subplots_adjust(wspace=0.2, hspace=0.5) #subplot 간 간격 조절
plt.show()

# 2.7 Family - SibSp(형제 자매) + Parch(부모, 자녀)


In [ ]:
#형제/자매, 부모/자녀 열을 합하여 FamilySize 열 만들기기
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] +1
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] +1

print("Maximum size of Family: ", df_train['FamilySize'].max())
print("Minimum size of Family: ", df_train['FamilySize'].min())

In [ ]:
f,ax = plt.subplots(1,3,figsize=(40,10))

#FamilySize별 탑승객 수 
sns.countplot(x='FamilySize', data=df_train, ax=ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded', y=1.02)

#FamilySize별 생존/사망인원수
sns.countplot(x='FamilySize', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('(2) Survived countplot depending on FamilySize', y=1.02)


#FamilySize에따른 생존비율
df_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=True
          ).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize', y=1.02)

plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

# 2.8 Fare
탑승요금

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))

#fare에 따른 분포 그래프프
g = sns.distplot(df_train['Fare'], color='b', label='Skewness : {:.2f}'.format(
    df_train['Fare'].skew()),ax=ax) #skew=왜도
g=g.legend(loc='best') #범례위치 자동지정


In [ ]:
#Fare열에 있는 결측값을 평균값으로 대체 
#데이터프레임.loc[조건] : 조건에 부한하는 행 선택
df_test.loc[df_test.Fare.isnull(),'Fare']=df_test['Fare'].mean()

df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)



In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
g = sns.distplot(df_train['Fare'], color='b', label='Skewness : {:.2f}'.format(
    df_train['Fare'].skew()), ax=ax)
g=g.legend(loc='best')

# 2.9 Cabin
null 비율이 약 80%이므로 모델에서제외

In [ ]:
df_train.head()

# 2.10 Ticket

In [ ]:
df_train['Ticket'].value_counts()